In [1]:
import math
import numpy as np

In [2]:
def KeplerEquation(e, M, precision_error):
    '''
    Solves Kepler's equation, for mean anomaly and eccentricity, 
    difference between values precision error.
    Note! It expects M in radians and returns the eccentric anomaly E in radians.
    M can be any value, it is normalized because of 2pi periodicity into the interval 
    [0, 2pi). It is expected to be bigger that 2pi if more than one orbital period passed 
    since perilehion. 
    Note! It might be better to be normalized as degrees and even as periods before 
    going to radians.
    '''
    # normalize M
    if ( M >= 2*math.pi ):
        two_pi = 2*math.pi
        M = M - two_pi*int(M / two_pi)
    
    # initial guess
    if e < 0.8:
        E = M
    else:
        E = math.pi
    
    prevE = math.inf
    while(abs(E-prevE)>precision_error):
        prevE = E
        cos_E = math.cos(E)
        E = (M - e*(E*cos_E - math.sin(E)))/(1-e*cos_E)

    return E


In [3]:
## backup

def reverse_X_rotation_matrix(angle):
    sin_a = math.sin(angle)
    cos_a = math.cos(angle)
    return [[1, 0,     0     ],
            [0, cos_a, -sin_a],
            [0, sin_a, cos_a ]]

def reverse_Z_rotation_matrix(angle):
    sin_a = math.sin(angle)
    cos_a = math.cos(angle)
    return [[cos_a, -sin_a, 0 ],
            [sin_a, cos_a,  0 ],
            [0,     0,      1 ]]

def deg2rad(deg):
    ''' make it radians '''
    return deg*math.pi/180

assert (deg2rad(90) == math.pi/2)
assert (deg2rad(180) == math.pi)
assert (deg2rad(60) == math.pi/3)


def rad2deg(rad):
    return rad*180/math.pi

assert ( 90  == rad2deg(math.pi/2))
assert ( abs(180 - rad2deg(math.pi)) < 1e-12 )
assert ( abs( 60  - rad2deg(math.pi/3)) < 1e-12 )


class EllipticKeplerOrbit :
    '''
    Orbital parameters for elliptic orbit, It contains all orbital parameters and
    it can calculate heliocentric eccliptic, heliocentric Earth equatorial, 
    and polar coordinates in orbital plane if needed.
    It uses only Kepler's laws.
    
    Mandatory Parameters List - Minor Planet Center File Data Format:
    ¶
        Epoch (in packed form, .0 TT)
        Mean anomaly at the epoch, in degrees
        Argument of perihelion, J2000.0 (degrees)
        Longitude of the ascending node, J2000.0 (degrees)
        Inclination to the ecliptic, J2000.0 (degrees)
        Orbital eccentricity
        Mean daily motion (degrees per day)
        Semimajor axis (AU)
    '''
    def __init__(self, epoch, mean_anomaly_epoch, argument_perihelion, lngt_ascending_node, inclination, eccentricity, mean_daily_motion, semi_major_axis, earth_axis_tilt = 23.4365472133):
        self.epoch = epoch
        self.m0    = mean_anomaly_epoch
        self.a     = semi_major_axis
        self.e     = eccentricity
        self.n     = mean_daily_motion
        ## semi-minor axis
        self.b     = self.a*math.sqrt(1-self.e*self.e)
        
        ## Transformation matrixes
        mtxRZ_omega = np.array( reverse_Z_rotation_matrix( deg2rad( argument_perihelion ) ) )
        mtxRX_inclination = np.array( reverse_X_rotation_matrix( deg2rad( inclination ) ) )
        mtxRZ_AsnNodeOmega = np.array( reverse_Z_rotation_matrix( deg2rad( lngt_ascending_node ) ) )
        
        ## Eccliptic Related
        self.mtxEccliptic =  mtxRZ_AsnNodeOmega.dot( mtxRX_inclination.dot( mtxRZ_omega ) )
        self.mtxGeoEquatorialFromEccliptic = np.array(reverse_X_rotation_matrix(deg2rad(earth_axis_tilt)))
        # Earth Equatorial
        self.mtxGeoEquatotial = self.mtxGeoEquatorialFromEccliptic.dot( self.mtxEccliptic )
        
    def matrixEcclipticCoords(self):
        return self.mtxEccliptic;
    
    def vectorsPQ_EcclipticCoords(self):
        return [self.mtxEccliptic[:,0], self.mtxEccliptic[:,1]];
        
    def matrixGeoEquatotialCoords(self):
        return self.mtxGeoEquatotial

    def vectorsPQ_GeoEquatotialCoords(self):
        return [self.mtxGeoEquatotial[:,0], self.mtxGeoEquatotial[:,1]];
        
    def kepler_orbit_position_in_orbital_cs(self, t):
        '''
        Calculate position at 2D cartesian or polar coordinates at orbital coordinate system
        Calculates position using Kepler's 1st and 2nd law.
        Orbit parameters
        a  - major semi-axis
        e  - eccentricity,
        n  - mean daily motion
        m0 - mean anomaly
        Arguments:
        t - time since epoch.
        Returns
        a pair of planet distance and angle true anomaly because the angle is not directly 
        needed and to avoid quadrant problem, sin(v) and cos(v) sin and cos from the true 
        anomaly are returned
        '''
        ## Mean anomaly in degrees
        t = t - self.epoch
        m = self.n*t + self.m0

        ## Normalize it at 360 degrees, 
        if ( m > 360 ):
            m = m - 360*int(m / 360)

        ## and make it radians
        m = deg2rad(m)

        ## Calculate Eccentric anomaly from the mean anomaly
        ecc_anomaly = KeplerEquation(self.e, m, 1e-6)

        cos_E = math.cos(ecc_anomaly)
        sin_E = math.sin(ecc_anomaly)
        
        ## Calculate carthesian coordinates
        xp = self.a*(cos_E-self.e)
        yp = self.b*sin_E

        ## return polar coordinates with the pole the ellipse focus, where the Sun is.
        return [xp, yp]

### P,Q Vector and only first 2 columns from the matrixes

Calculating coordinates to ecliptic plane or Earth equatorial plane is done by multiplication with matrix 
$$
\left(\begin{array}{cc} x_{g,eq}\\ y_{g,eq}\\ z_{g,eq} \\ \end{array}\right) =
    M_{ecc/g,eq} 
    \cdot
    \left(\begin{array}{cc} x_p\\ y_p\\ 0 \\\end{array}\right)
$$

Because the Z coordinate of the position vector of the planet in the initial coordinate system is always 0. then we do not need to know the 3rd column of that matrix only first 2 are enough. So we may reduce the matrixes to a matrix with dimentions $3 \times 2$

If we note reach column with letters, denoting each column as a vector.
$$
M_{ecc/g,eq} = \left(\begin{array}{cc} 
    P & Q & R 
    \end{array}\right)
$$

Then the result is

$$
\left(\begin{array}{cc} x_{g,eq}\\ y_{g,eq}\\ z_{g,eq} \\ \end{array}\right) =
    \left(\begin{array}{cc} 
    P & Q 
    \end{array}\right)
    \cdot
    \left(\begin{array}{cc} x_p\\ y_p \\\end{array}\right)
$$

or in more convenient form, if the vector form

$$
\left(\begin{array}{cc} x_{g,eq}\\ y_{g,eq}\\ z_{g,eq} \\ \end{array}\right) =
    x_pP +
    y_pQ 
$$


### Unit test for creation of matrixes.

In [4]:
#Test coorinate convertion matrixes.
# Data is taken from Minor Planet Center, the coeficients of the matrixes are also provided by them 

eccentricity = 0.7959518780556988
semi_major_axis = 1.340551489041067
mean_motion = 0.6350074562624759

inclination = 31.43144208629369
longitude_ascending_node = 213.071641162073
argument_perihelion = 233.4372674009908

# not used because we have 
#mean anomaly 138.5679892519656
time_of_perihelion_passage = 2458982.285256400691

elkOrbit = EllipticKeplerOrbit(time_of_perihelion_passage, 0, argument_perihelion, longitude_ascending_node, inclination, eccentricity, mean_motion, semi_major_axis)

# Heliocentric, Eccliptic Plane.
print("Ecclitic")
print(elkOrbit.matrixEcclipticCoords())

# Geo-Equatorial
print()
print("Geo-Equatorial")
print(elkOrbit.matrixGeoEquatotialCoords())


Ecclitic
[[ 0.12520723 -0.95044545 -0.28456388]
 [ 0.89938016 -0.01235574  0.43699274]
 [-0.41885376 -0.31064576  0.85326476]]

Geo-Equatorial
[[ 0.12520723 -0.95044545 -0.28456388]
 [ 0.99177435  0.11221773  0.06156959]
 [-0.02658542 -0.28993211  0.95667789]]


### Minor Planet Center provides the matrix for Geo-Equatorial transformation 

Minor Planet Center provides coeficients of P and Q vectors, so we could check them for our test asteroid
[153002](https://www.minorplanetcenter.net/db_search/show_object?utf8=%E2%9C%93&object_id=153002)

#### P-vector [x]	0.12540215  
#### P-vector [y]	0.99175374  
#### P-vector [z]	-0.02643518 
#### Q-vector [x]	-0.95021191  
#### Q-vector [y]	0.11240279  
#### Q-vector [z]	-0.29062507 



The unit test just checks those values, more such tests are listed later with different cases, especially with inclination bigger than 90 degrees.

In [5]:
# PQ vectors, Eccliptic Plane.
print("PQ vectors Ecclitic")
trnPQVect = elkOrbit.vectorsPQ_EcclipticCoords() 
print(f"P={trnPQVect[0]}")
print(f"O={trnPQVect[1]}")

# PQ vectors Geo-Equatorial - Checked with Minor Planet Center.
print()
print("PQ vectors Geo-Equatorial - Checked with Minor Planet Center. ")
trnPQVect = elkOrbit.vectorsPQ_GeoEquatotialCoords()
print(f"P={trnPQVect[0]}")
print(f"O={trnPQVect[1]}")

# P Vector
assert(abs(trnPQVect[0][0]-0.12540215)<0.0005)
assert(abs(trnPQVect[0][1]-0.99175374)<0.0005)
assert(abs(trnPQVect[0][2]-(-0.02643518))<0.0005)
print("P-vector checked")

# P Vector
assert(abs(trnPQVect[1][0]-(-0.95021191))<0.0005)
assert(abs(trnPQVect[1][1]-0.11240279)<0.0005)
assert(abs(trnPQVect[1][2]-(-0.29062507))<0.005)
print("Q-vector checked")

PQ vectors Ecclitic
P=[ 0.12520723  0.89938016 -0.41885376]
O=[-0.95044545 -0.01235574 -0.31064576]

PQ vectors Geo-Equatorial - Checked with Minor Planet Center. 
P=[ 0.12520723  0.99177435 -0.02658542]
O=[-0.95044545  0.11221773 -0.28993211]
P-vector checked
Q-vector checked


### Regression test
It is needed to be rechecked and it passed. Because the code was organized into a class and there might be mistakes. Regression test is passing.

In [6]:
#Checking for some period with one month difference
tp = time_of_perihelion_passage
## DATA from JPL
dates_JD_and_distances = []
## 2020-05-12 00:00 UTC
dates_JD_and_distances.append([2458981.5, 0.269]); ## just before perihelion
## 2020-05-12 00:00 UTC
dates_JD_and_distances.append([tp, .2735370137084662]); ## perihelion
## 2020-05-13 00:00 UTC
dates_JD_and_distances.append([2458982.5, 0.268]); ## just after perihelion
## 2020-05-14 00:00 UTC
dates_JD_and_distances.append([2458983.5, 0.269]); ## just after perihelion
## 2020-05-19 00:00 UTC
dates_JD_and_distances.append([2458988.5, 0.32]); ## days after perihelion
## 2020-06-19 00:00 UTC
dates_JD_and_distances.append([2459019.5, 0.888]);
## 2020-07-19 00:00 UTC
dates_JD_and_distances.append([2459049.5, 1.309]);
## 2020-08-19 00:00 UTC
dates_JD_and_distances.append([2459080.5, 1.64]);
## 2020-09-19 00:00 UTC
dates_JD_and_distances.append([2459111.5, 1.894]);
## 2020-10-19 00:00 UTC
dates_JD_and_distances.append([2459141.5, 2.084]);
## 2020-11-19 00:00 UTC
dates_JD_and_distances.append([2459172.5, 2.23]);
## 2020-12-19 00:00 UTC
dates_JD_and_distances.append([2459202.5, 2.329]);
## 2021-01-19 00:00 UTC
dates_JD_and_distances.append([2459233.5, 2.39]);
## 2021-02-16 00:00 UTC
dates_JD_and_distances.append([2459261.5, 2.411]);
## 2021-02-19 00:00 UTC
dates_JD_and_distances.append([2459264.5, 2.412]);
## 2021-03-19 00:00 UTC
dates_JD_and_distances.append([2459292.5, 2.397]);
## 2021-04-19 00:00 UTC
dates_JD_and_distances.append([2459323.5, 2.344]);
## 2021-05-19 00:00 UTC
dates_JD_and_distances.append([2459353.5, 2.253]);
## 2021-06-19 00:00 UTC
dates_JD_and_distances.append([2459384.5, 2.116]);
# 2021-07-19 00:00 UTC
dates_JD_and_distances.append([2459414.5, 1.936]);
# 2021-08-19 00:00 UTC
dates_JD_and_distances.append([2459445.5, 1.694]);
# 2021-09-19 00:00 UTC
dates_JD_and_distances.append([2459476.5, 1.380]);
# 2021-10-03 00:00 UTC 
dates_JD_and_distances.append([2459490.5, 1.208]);
# 2021-10-10 00:00 UTC 
dates_JD_and_distances.append([2459497.5, 1.114]);
# 2021-10-19 00:00 UTC
dates_JD_and_distances.append([2459506.5, 0.983]);
# 2021-10-23 00:00 UTC
dates_JD_and_distances.append([2459510.5, 0.92]);
# 2021-11-19 00:00 UTC
dates_JD_and_distances.append([2459537.5, 0.431]);
# 2021-11-27 00:00 UTC
dates_JD_and_distances.append([2459545.5, 0.297]);
# 2021-11-29 00:00 UTC
dates_JD_and_distances.append([2459547.5, 0.278]);
# 2021-11-30 00:00 UTC
dates_JD_and_distances.append([2459548.5, 0.273]);
# 2021-12-19 00:00 UTC
dates_JD_and_distances.append([2459567.5, 0.544]);
# 2022-01-19 00:00 UTC
dates_JD_and_distances.append([2459598.5, 1.068]);
# 2022-02-19 00:00 UTC
dates_JD_and_distances.append([2459629.5, 1.455]);
# 2022-03-19 00:00 UTC
dates_JD_and_distances.append([2459657.5, 1.725]);


In [7]:
# recheck the code for 2D coordinates.
for dd in dates_JD_and_distances:
    today    = dd[0]
    exp_dist = dd[1]
    rp = elkOrbit.kepler_orbit_position_in_orbital_cs(today)
    r = math.sqrt(rp[0]*rp[0] + rp[1]*rp[1])
    if ((today-time_of_perihelion_passage) == 0):
        print (f"perihelion exp: {exp_dist:.3f} AU, calc: {r:.10f} AU, diff: {abs(exp_dist - r):.10f}")
    else:
        print (f"{today}, exp: {exp_dist:.3f} AU, calc: {r:.10f} AU, diff: {abs(exp_dist - r):.10f}")


2458981.5, exp: 0.269 AU, calc: 0.2745051146 AU, diff: 0.0055051146
perihelion exp: 0.274 AU, calc: 0.2735370137 AU, diff: 0.0000000000
2458982.5, exp: 0.268 AU, calc: 0.2736095819 AU, diff: 0.0056095819
2458983.5, exp: 0.269 AU, calc: 0.2758456712 AU, diff: 0.0068456712
2458988.5, exp: 0.320 AU, calc: 0.3266874008 AU, diff: 0.0066874008
2459019.5, exp: 0.888 AU, calc: 0.8886835768 AU, diff: 0.0006835768
2459049.5, exp: 1.309 AU, calc: 1.3082578613 AU, diff: 0.0007421387
2459080.5, exp: 1.640 AU, calc: 1.6376679373 AU, diff: 0.0023320627
2459111.5, exp: 1.894 AU, calc: 1.8917239261 AU, diff: 0.0022760739
2459141.5, exp: 2.084 AU, calc: 2.0809662265 AU, diff: 0.0030337735
2459172.5, exp: 2.230 AU, calc: 2.2269586195 AU, diff: 0.0030413805
2459202.5, exp: 2.329 AU, calc: 2.3254802144 AU, diff: 0.0035197856
2459233.5, exp: 2.390 AU, calc: 2.3863852525 AU, diff: 0.0036147475
2459261.5, exp: 2.411 AU, calc: 2.4071995743 AU, diff: 0.0038004257
2459264.5, exp: 2.412 AU, calc: 2.4075343965 AU,